In [9]:
import torch
import torch.onnx as onnx
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision


In [10]:
training_data = torchvision.datasets.MNIST(
    '/files/',
    train=True,
    download=True,
    transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                              torchvision.transforms.Normalize((0.1307,), (0.3081,))]
                                             )
)

In [11]:
train_dataloader = DataLoader(training_data, 64, shuffle=True)
epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10
batch_size = 64

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [22]:
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [23]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([10, 1, 5, 5])
conv1.bias 	 torch.Size([10])
conv2.weight 	 torch.Size([20, 10, 5, 5])
conv2.bias 	 torch.Size([20])
fc1.weight 	 torch.Size([50, 320])
fc1.bias 	 torch.Size([50])
fc2.weight 	 torch.Size([10, 50])
fc2.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7]}]


In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (data, target) in enumerate(train_dataloader):
        # Compute prediction and loss
        pred = model(data)
        loss = loss_fn(pred, target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(data)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [25]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.341269  [    0/60000]


C:\Users\Alex Vogt\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 2.257834  [ 6400/60000]
loss: 2.137221  [12800/60000]
loss: 1.808013  [19200/60000]
loss: 1.140869  [25600/60000]
loss: 1.221116  [32000/60000]
loss: 0.827101  [38400/60000]
loss: 0.688913  [44800/60000]
loss: 0.800475  [51200/60000]
loss: 0.722362  [57600/60000]
Epoch 2
-------------------------------
loss: 0.362405  [    0/60000]
loss: 0.823674  [ 6400/60000]
loss: 0.541458  [12800/60000]
loss: 0.450030  [19200/60000]
loss: 0.604892  [25600/60000]
loss: 0.754719  [32000/60000]
loss: 0.436186  [38400/60000]
loss: 0.382034  [44800/60000]
loss: 0.431634  [51200/60000]
loss: 0.513731  [57600/60000]
Epoch 3
-------------------------------
loss: 0.545189  [    0/60000]
loss: 0.471490  [ 6400/60000]
loss: 0.471436  [12800/60000]
loss: 0.425937  [19200/60000]
loss: 0.787947  [25600/60000]
loss: 0.322682  [32000/60000]
loss: 0.402482  [38400/60000]
loss: 0.310229  [44800/60000]
loss: 0.416072  [51200/60000]
loss: 0.403630  [57600/60000]
Done!


In [26]:
torch.save(model.state_dict(), 'model_weights.pth')

Load:

the_model = TheModelClass(*args, **kwargs)
the_model.load_state_dict(torch.load(PATH))